In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pickle
from backgammon import *
from elements import *
from tqdm import tqdm
from ai_strategies import *
import random

In [3]:
with open('data/testing.pkl', 'r') as f:
    test_data = pickle.load(f)

def test(plays, w):
    pos = 0 
    c = 0
    for p in tqdm(plays):
        board, t, poss_moves, cl, moves = p
        if len(poss_moves)<=1:
            continue
        c+=1
        probs = []
        for j, ms in enumerate(poss_moves):
            temp = board.copy()
            for m in ms:
                temp = temp.move(t.player, *m)
            f = temp.feat(board, t.player)
            if j == cl:
                f_j = f
            probs.append(score(w, f))
        probs = softmax(probs)
        if probs[cl] == max(probs): 
            pos+=1
    return float(pos)/c

def softmax(s):
    num = [np.exp(s[i])+0.00001 for i in range(len(s))]
    return [x/sum(num) for x in num]

In [19]:
def play_n(champion, challenger, n, show=False):
    champ_score = 0
    chal_score = 0
    while champ_score<n and chal_score<n:
        col, winner_strat, mars = Game().computer_vs_computer_play(champion, challenger, False)
        if winner_strat.name == champion.name:
            if mars:
                champ_score+=2
            else:
                champ_score+=1
        elif winner_strat.name == challenger.name:
            if mars:
                chal_score+=2
            else:
                chal_score+=1
    if show:
        print 'Champ: {}'.format(champ_score)
        print 'Chal: {}'.format(chal_score)
    if champ_score >= n:
        return 1, abs(champ_score-chal_score)
    else:
        return 0, abs(champ_score-chal_score)

def generate_new_challenger(champion):
    new_w = champion.asList[:]
    for i in range(len(new_w)):
        new_w[i] += random.uniform(-0.5,0.5)
    return SimpleWeights(new_w, True,'challenger')

def genetic_udpate(init_champion, alpha, rounds, max_champ, max_it, past=None):
    if past is None:
        champions=[init_champion]
        l = 0
    else:
        champions = past
        l=len(champions)
    it = 0
    n_champs = 0
    killer_tour = 0
    changes = []
    c = 0
    while n_champs<max_champ and it<max_it:
        it+=1
        if it % 25 == 0 and it>0:
            print "{} iterations passed...".format(it)
        challenger = generate_new_challenger(champions[-1])
        res, diff = play_n(champions[-1], challenger, rounds)
        if it % 500 == 0:
            changes.append(float(c)/500)
            c=0
        if res == 1:
            continue
        elif res == 0:
#             print "We are getting a new Champion !"
            n_champs+=1
            c+=1
            a = alpha
            if diff >= rounds/2:
                a*=2
            new_champ_weights = [(1-a)*w[0]+a*w[1] for w in zip(champions[-1].asList, challenger.asList)]
            champions.append(SimpleWeights(new_champ_weights, True, 'champion_{}'.format(l+n_champs)))
    return champions, changes

In [5]:
champion_21 = logistic.copy()

In [20]:
champs_21, changes = genetic_udpate(champion_21, 0.1, 15, float("inf"), 1000, champs_21)

25 iterations passed...
50 iterations passed...
75 iterations passed...
100 iterations passed...
125 iterations passed...
150 iterations passed...
175 iterations passed...
200 iterations passed...
225 iterations passed...
250 iterations passed...
275 iterations passed...
300 iterations passed...
325 iterations passed...
350 iterations passed...
375 iterations passed...
400 iterations passed...
425 iterations passed...
450 iterations passed...
475 iterations passed...
500 iterations passed...
525 iterations passed...
550 iterations passed...
575 iterations passed...
600 iterations passed...
625 iterations passed...
650 iterations passed...
675 iterations passed...
700 iterations passed...
725 iterations passed...
750 iterations passed...
775 iterations passed...
800 iterations passed...
825 iterations passed...
850 iterations passed...
875 iterations passed...
900 iterations passed...
925 iterations passed...
950 iterations passed...
975 iterations passed...
1000 iterations passed...


In [21]:
play_n(champs_21[-1], logistic, 21, True)

Champ: 21
Chal: 11


(1, 10)

In [18]:
changes

[]

In [22]:
test(test_data, champs_21[-1])

100%|██████████| 2677/2677 [00:53<00:00, 49.97it/s]


0.6855713094245204

In [23]:
with open('data/champs_genetic_21.pkl', 'w') as f:
    pickle.dump(champs_21, f)

# NEW FEAT

In [25]:
loglong = logistic_long.copy()

In [26]:
champs_41 = [loglong] 

In [34]:
champs_41, changes_41 = genetic_udpate(loglong, 0.1, 12, float("inf"), 1000, champs_41)

25 iterations passed...
50 iterations passed...
75 iterations passed...
100 iterations passed...
125 iterations passed...
150 iterations passed...
175 iterations passed...
200 iterations passed...
225 iterations passed...
250 iterations passed...
275 iterations passed...
300 iterations passed...
325 iterations passed...
350 iterations passed...
375 iterations passed...
400 iterations passed...
425 iterations passed...
450 iterations passed...
475 iterations passed...
500 iterations passed...
525 iterations passed...
550 iterations passed...
575 iterations passed...
600 iterations passed...
625 iterations passed...
650 iterations passed...
675 iterations passed...
700 iterations passed...
725 iterations passed...
750 iterations passed...
775 iterations passed...
800 iterations passed...
825 iterations passed...
850 iterations passed...
875 iterations passed...
900 iterations passed...
925 iterations passed...
950 iterations passed...
975 iterations passed...
1000 iterations passed...


In [35]:
champs_41, changes_41 = genetic_udpate(loglong, 0.1, 12, float("inf"), 1000, champs_41)

25 iterations passed...
50 iterations passed...
75 iterations passed...
100 iterations passed...
125 iterations passed...
150 iterations passed...
175 iterations passed...
200 iterations passed...
225 iterations passed...
250 iterations passed...
275 iterations passed...
300 iterations passed...
325 iterations passed...
350 iterations passed...
375 iterations passed...
400 iterations passed...
425 iterations passed...
450 iterations passed...
475 iterations passed...
500 iterations passed...
525 iterations passed...
550 iterations passed...
575 iterations passed...
600 iterations passed...
625 iterations passed...
650 iterations passed...
675 iterations passed...
700 iterations passed...
725 iterations passed...
750 iterations passed...
775 iterations passed...
800 iterations passed...
825 iterations passed...
850 iterations passed...
875 iterations passed...
900 iterations passed...
925 iterations passed...
950 iterations passed...
975 iterations passed...
1000 iterations passed...


In [36]:
champs_41, changes_41 = genetic_udpate(loglong, 0.1, 12, float("inf"), 250, champs_41)

25 iterations passed...
50 iterations passed...
75 iterations passed...
100 iterations passed...
125 iterations passed...
150 iterations passed...
175 iterations passed...
200 iterations passed...
225 iterations passed...
250 iterations passed...


In [42]:
champs_41, changes_41 = genetic_udpate(loglong, 0.1, 12, float("inf"), 250, champs_41)

25 iterations passed...
50 iterations passed...
75 iterations passed...
100 iterations passed...
125 iterations passed...
150 iterations passed...
175 iterations passed...
200 iterations passed...
225 iterations passed...
250 iterations passed...


In [43]:
champs_41, changes_41 = genetic_udpate(loglong, 0.1, 12, float("inf"), 500, champs_41)

25 iterations passed...
50 iterations passed...
75 iterations passed...
100 iterations passed...
125 iterations passed...
150 iterations passed...
175 iterations passed...
200 iterations passed...
225 iterations passed...
250 iterations passed...
275 iterations passed...
300 iterations passed...
325 iterations passed...
350 iterations passed...
375 iterations passed...
400 iterations passed...
425 iterations passed...
450 iterations passed...
475 iterations passed...
500 iterations passed...


In [47]:
champs_41_2 = champs_41[:]
champs_41_2, changes_41 = genetic_udpate(loglong, 0.1, 12, float("inf"), 750, champs_41_2)

25 iterations passed...
50 iterations passed...
75 iterations passed...
100 iterations passed...
125 iterations passed...
150 iterations passed...
175 iterations passed...
200 iterations passed...
225 iterations passed...
250 iterations passed...
275 iterations passed...
300 iterations passed...
325 iterations passed...
350 iterations passed...
375 iterations passed...
400 iterations passed...
425 iterations passed...
450 iterations passed...
475 iterations passed...
500 iterations passed...
525 iterations passed...
550 iterations passed...
575 iterations passed...
600 iterations passed...
625 iterations passed...
650 iterations passed...
675 iterations passed...
700 iterations passed...
725 iterations passed...
750 iterations passed...


In [100]:
len(champs_41)

1310

In [96]:
test(test_data, champs_21[-1])

100%|██████████| 2677/2677 [00:55<00:00, 48.40it/s]


0.2585487906588824

In [98]:
test(test_data, ql)

  1%|          | 16/2677 [00:00<01:15, 35.32it/s]/Users/virgileaudi/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
100%|██████████| 2677/2677 [00:54<00:00, 48.78it/s]


0.1780650542118432

In [49]:
play_n(champs_41[-1], loglong, 21, True)

Champ: 21
Chal: 3


(1, 18)

In [50]:
with open('data/champs_genetic_41.pkl', 'w') as f:
    pickle.dump(champs_41, f)

In [51]:
! open data

In [52]:
play_n(champs_41[-1], loglong, 15, True)

Champ: 16
Chal: 0


(1, 16)

In [53]:
play_n(champs_41[-1], logistic, 15, True)

Champ: 15
Chal: 0


(1, 15)

In [54]:
play_n(loglong, logistic, 15, True)

Champ: 15
Chal: 1


(1, 14)

In [55]:
play_n(champs_21[-1], logistic, 15, True)

Champ: 15
Chal: 5


(1, 10)

In [57]:
play_n(champs_21[-1], loglong, 15, True)

Champ: 15
Chal: 5


(1, 10)

In [58]:
play_n(champs_21[-1], champs_41[-1], 15, True)

Champ: 8
Chal: 15


(0, 7)

In [59]:
ql = SimpleWeights([10.513928453727098, 0.03665897636716714, -10.383521078312361, -1.9812681320846859, -7.2268173161494094, -7.1934974631247792, -0.058559108604641211, 0.14603343061814536, -0.15687361404182876, -1.5940895247508318, -1.1278465977883321, -0.1358589500191868, 0.46214445743694715, 1.2156356424217336, -0.55597493779631557, -1.3658852329825804, -0.12275466238407588, 0.10202476751770939, -1.4574258604994315, -0.23711859754993708, 1.0878489704323944, -0.037370307502920044, -40.93975833543071, -3.3731213853774831, -17.443373294193606, -11.285669866742879, -0.059228216559662754, -4.6652175406332015, -3.663706250577957, -3.7849705874744868, -1.5357370825787495, 0.041129684329487506, -1.8851695285103183, -1.2146105759529324, -1.634758410668683, -2.169318540721386, -0.027724832764959715, -0.55917520234064622, -2.8149248399111966, -2.4056688743066532, 0.40174027284356889], True, 'ql')

In [60]:
play_n(ql, champs_41[-1], 15, True)

Champ: 0
Chal: 15


(0, 15)

In [61]:
play_n(ql, champs_21[-1], 15, True)

Champ: 1
Chal: 16


(0, 15)

In [65]:
play_n(ql, loglong, 15, True)

Champ: 4
Chal: 15


(0, 11)

In [64]:
play_n(ql, logistic, 15, True)

Champ: 15
Chal: 8


(1, 7)